    Esse projeto tem a função de:

    1-Criar um sistema TKINTER que:
   

        conforme link https://www.researchgate.net/figure/Figura-2-Simbolos-para-fluxograma-Fonte-Os-autores_fig1_325917131

        
    2-Deve ter as seguintes opções de input de atividades: 

    Atividade = Retângulo horizontal

    Inspeção = Círculo

    Adiamento = D semicírculo ou símbolo de espera

    Armazenagem = Triângulo invertido

    Decisão = Losango

    Início = Oval ou elipse

    Fim = Oval ou elipse

    Fluxo = Linha ou seta

    Documento = Retângulo com uma linha ondulada na parte inferior

    Nome do banco de dados = Cilindro

    Movimentação = Linha ou seta contínua

    3-As opções de ipnut terá a função de armazenar qualquer nome aleatório que será escrito manualmente e também uma text box adicional para escolher uma das opções de Input;

    4-O sistema TKINTER deverá ter um botão de confirmação que vai gerar uma imagem no próprio sistema TKINTER do workflow ou fluxograma alimentados com as informações incluídas em opções de Input;

    4B-A visualização do Fluxograma ou workflow deve seguir uma vizualização de fácil entendimento humano, ou seja o início começa na esquerda e o fim na direita, podendo ser de cima para baixo ou 
    a melhor forma de visualização que você puder colocar mas atendendo a lógica que o começo é na esquerda e o fim na direita.

    5-O sistema TKINTER deverá ter a função de criar uma descrição do problema a ser resolvido com base nas informações incluídas no input de atividades, criando assim um texto automático,
    para que eu possa ler qual é o problema a ser resolvido;

    6-Deve ter um botão para limpar as informações do sistema;

    7-Deve ter um botão para escolher onde será incluido o elemento será disposto no canvas, se é na esquerda, direita abaixo ou acima do último canvas inserido, com exceção do elemento de inicio que sempre vai começar na esquerda no topo do canvas;

    8-A conexão vai ser feita na forma de uma linha ou seta (Fluxo) sem mover os elementos.

In [4]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import math
from PIL import Image, ImageTk, ImageDraw
import io

class ElementoFluxograma:
    def __init__(self, tipo, nome, canvas_id=None, x=0, y=0, width=100, height=50):
        self.tipo = tipo
        self.nome = nome
        self.canvas_id = canvas_id
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.conexoes = []  # Lista de ElementoFluxograma conectados a este

class SistemaFluxograma(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Sistema de Fluxograma")
        self.geometry("1200x800")
        self.elementos = []
        self.conexoes = []
        self.ultimo_elemento = None
        self.configurar_ui()
        
    def configurar_ui(self):
        # Frame principal dividido em duas partes
        main_frame = ttk.Frame(self)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Frame esquerdo para inputs e controles
        left_frame = ttk.LabelFrame(main_frame, text="Controles")
        left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=False, padx=5, pady=5)
        
        # Frame direito para o canvas de visualização
        right_frame = ttk.LabelFrame(main_frame, text="Visualização do Fluxograma")
        right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Criação dos controles no frame esquerdo
        ttk.Label(left_frame, text="Nome do Elemento:").pack(padx=5, pady=5, anchor=tk.W)
        self.nome_elemento = ttk.Entry(left_frame, width=30)
        self.nome_elemento.pack(padx=5, pady=5, fill=tk.X)
        
        ttk.Label(left_frame, text="Tipo do Elemento:").pack(padx=5, pady=5, anchor=tk.W)
        self.tipos_elementos = [
            "Início", "Atividade", "Inspeção", "Adiamento", "Armazenagem", 
            "Decisão", "Documento", "Banco de Dados", "Fim"
        ]
        self.tipo_elemento = ttk.Combobox(left_frame, values=self.tipos_elementos, state="readonly")
        self.tipo_elemento.current(0)
        self.tipo_elemento.pack(padx=5, pady=5, fill=tk.X)
        
        ttk.Label(left_frame, text="Posição do Elemento:").pack(padx=5, pady=5, anchor=tk.W)
        self.posicoes = ["Direita", "Abaixo", "Esquerda", "Acima"]
        self.posicao_elemento = ttk.Combobox(left_frame, values=self.posicoes, state="readonly")
        self.posicao_elemento.current(0)
        self.posicao_elemento.pack(padx=5, pady=5, fill=tk.X)
        
        # Botões de ação
        buttons_frame = ttk.Frame(left_frame)
        buttons_frame.pack(padx=5, pady=10, fill=tk.X)
        
        self.adicionar_btn = ttk.Button(buttons_frame, text="Adicionar Elemento", command=self.adicionar_elemento)
        self.adicionar_btn.pack(side=tk.LEFT, padx=5)
        
        self.conectar_btn = ttk.Button(buttons_frame, text="Conectar Elementos", command=self.mostrar_dialogo_conexao)
        self.conectar_btn.pack(side=tk.LEFT, padx=5)
        
        self.limpar_btn = ttk.Button(buttons_frame, text="Limpar Tudo", command=self.limpar_tudo)
        self.limpar_btn.pack(side=tk.LEFT, padx=5)
        
        self.gerar_descricao_btn = ttk.Button(left_frame, text="Gerar Descrição", command=self.gerar_descricao)
        self.gerar_descricao_btn.pack(padx=5, pady=5, fill=tk.X)
        
        # Área para a descrição gerada
        ttk.Label(left_frame, text="Descrição do Fluxograma:").pack(padx=5, pady=5, anchor=tk.W)
        self.descricao_text = scrolledtext.ScrolledText(left_frame, width=30, height=10, wrap=tk.WORD)
        self.descricao_text.pack(padx=5, pady=5, fill=tk.BOTH, expand=True)
        
        # Canvas para desenhar o fluxograma
        self.canvas_frame = ttk.Frame(right_frame)
        self.canvas_frame.pack(fill=tk.BOTH, expand=True)
        
        self.canvas = tk.Canvas(self.canvas_frame, bg="white")
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Adicionar scrollbars para o canvas
        h_scrollbar = ttk.Scrollbar(self.canvas_frame, orient=tk.HORIZONTAL, command=self.canvas.xview)
        h_scrollbar.pack(side=tk.BOTTOM, fill=tk.X)
        
        v_scrollbar = ttk.Scrollbar(self.canvas_frame, orient=tk.VERTICAL, command=self.canvas.yview)
        v_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.canvas.configure(xscrollcommand=h_scrollbar.set, yscrollcommand=v_scrollbar.set)
        self.canvas.bind("<Configure>", self.on_canvas_configure)
        
        # Inicializar variáveis de controle do canvas
        self.canvas_width = 2000
        self.canvas_height = 1000
        self.margin_x = 50
        self.margin_y = 50
        self.canvas.configure(scrollregion=(0, 0, self.canvas_width, self.canvas_height))
    
    def on_canvas_configure(self, event):
        self.desenhar_fluxograma()
    
    def adicionar_elemento(self):
        nome = self.nome_elemento.get().strip()
        tipo = self.tipo_elemento.get()
        posicao = self.posicao_elemento.get()
        
        if not nome:
            messagebox.showerror("Erro", "Por favor, insira um nome para o elemento.")
            return
        
        # Definir posição inicial para o primeiro elemento (Início)
        if not self.elementos:
            if tipo != "Início":
                messagebox.showerror("Erro", "O primeiro elemento deve ser do tipo 'Início'.")
                return
            x, y = self.margin_x, self.margin_y
        else:
            # Calcular posição com base no último elemento e na posição escolhida
            ultimo = self.ultimo_elemento
            spacing_x = 150
            spacing_y = 100
            
            if posicao == "Direita":
                x = ultimo.x + ultimo.width + spacing_x
                y = ultimo.y
            elif posicao == "Abaixo":
                x = ultimo.x
                y = ultimo.y + ultimo.height + spacing_y
            elif posicao == "Esquerda":
                x = max(self.margin_x, ultimo.x - spacing_x - 100)  # Evitar posições negativas
                y = ultimo.y
            elif posicao == "Acima":
                x = ultimo.x
                y = max(self.margin_y, ultimo.y - spacing_y - 50)  # Evitar posições negativas
        
        # Criar novo elemento
        elemento = ElementoFluxograma(tipo, nome, x=x, y=y)
        self.elementos.append(elemento)
        self.ultimo_elemento = elemento
        
        # Redesenhar o fluxograma
        self.desenhar_fluxograma()
        
        # Limpar o campo de nome após adicionar
        self.nome_elemento.delete(0, tk.END)
    
    def mostrar_dialogo_conexao(self):
        if len(self.elementos) < 2:
            messagebox.showerror("Erro", "É necessário ter pelo menos dois elementos para criar uma conexão.")
            return
        
        # Criar janela de diálogo
        dialog = tk.Toplevel(self)
        dialog.title("Conectar Elementos")
        dialog.geometry("300x200")
        dialog.transient(self)
        dialog.grab_set()
        
        ttk.Label(dialog, text="De:").pack(padx=5, pady=5)
        
        # Lista de nomes de elementos para origem
        nomes_elementos = [f"{i+1}. {elem.nome} ({elem.tipo})" for i, elem in enumerate(self.elementos)]
        origem_combo = ttk.Combobox(dialog, values=nomes_elementos, state="readonly")
        origem_combo.pack(padx=5, pady=5, fill=tk.X)
        origem_combo.current(0)
        
        ttk.Label(dialog, text="Para:").pack(padx=5, pady=5)
        
        # Lista de nomes de elementos para destino
        destino_combo = ttk.Combobox(dialog, values=nomes_elementos, state="readonly")
        destino_combo.pack(padx=5, pady=5, fill=tk.X)
        if len(nomes_elementos) > 1:
            destino_combo.current(1)
        
        def criar_conexao():
            try:
                origem_idx = origem_combo.current()
                destino_idx = destino_combo.current()
                
                if origem_idx == destino_idx:
                    messagebox.showerror("Erro", "Origem e destino não podem ser o mesmo elemento.")
                    return
                
                # Verificar se a conexão já existe
                origem = self.elementos[origem_idx]
                destino = self.elementos[destino_idx]
                
                for conexao in self.conexoes:
                    if conexao[0] == origem and conexao[1] == destino:
                        messagebox.showerror("Erro", "Esta conexão já existe.")
                        return
                
                # Adicionar conexão
                self.conexoes.append((origem, destino))
                origem.conexoes.append(destino)
                
                # Redesenhar o fluxograma
                self.desenhar_fluxograma()
                dialog.destroy()
                
            except Exception as e:
                messagebox.showerror("Erro", f"Erro ao criar conexão: {str(e)}")
        
        ttk.Button(dialog, text="Conectar", command=criar_conexao).pack(pady=20)
    
    def desenhar_fluxograma(self):
        # Limpar o canvas
        self.canvas.delete("all")
        
        # Desenhar elementos
        for elemento in self.elementos:
            self.desenhar_elemento(elemento)
        
        # Desenhar conexões
        for origem, destino in self.conexoes:
            self.desenhar_conexao(origem, destino)
    
    def desenhar_elemento(self, elemento):
        x, y = elemento.x, elemento.y
        width, height = elemento.width, elemento.height
        
        # Desenhar forma com base no tipo
        if elemento.tipo == "Atividade":
            # Retângulo horizontal
            elemento.canvas_id = self.canvas.create_rectangle(x, y, x + width, y + height, fill="lightblue")
        
        elif elemento.tipo == "Inspeção":
            # Círculo
            radius = min(width, height) / 2
            elemento.canvas_id = self.canvas.create_oval(x, y, x + width, y + height, fill="lightgreen")
        
        elif elemento.tipo == "Adiamento":
            # Semicírculo (D)
            elemento.canvas_id = self.canvas.create_arc(x, y, x + width, y + height, start=270, extent=180, fill="orange")
        
        elif elemento.tipo == "Armazenagem":
            # Triângulo invertido
            elemento.canvas_id = self.canvas.create_polygon(
                x, y, 
                x + width, y, 
                x + width/2, y + height, 
                fill="yellow"
            )
        
        elif elemento.tipo == "Decisão":
            # Losango
            elemento.canvas_id = self.canvas.create_polygon(
                x + width/2, y,
                x + width, y + height/2,
                x + width/2, y + height,
                x, y + height/2,
                fill="lightpink"
            )
        
        elif elemento.tipo in ["Início", "Fim"]:
            # Oval ou elipse
            cor = "lightgreen" if elemento.tipo == "Início" else "salmon"
            elemento.canvas_id = self.canvas.create_oval(x, y, x + width, y + height, fill=cor)
        
        elif elemento.tipo == "Documento":
            # Retângulo com linha ondulada na parte inferior
            elemento.canvas_id = self.canvas.create_rectangle(x, y, x + width, y + height, fill="lightyellow")
            # Linha ondulada (simplificada)
            wave_points = []
            wave_height = 10
            segments = 8
            for i in range(segments + 1):
                segment_x = x + (width * i / segments)
                segment_y = y + height + (wave_height * (i % 2))
                wave_points.append(segment_x)
                wave_points.append(segment_y)
            
            self.canvas.create_line(wave_points, smooth=True, fill="black")
        
        elif elemento.tipo == "Banco de Dados":
            # Cilindro
            top_height = height * 0.2
            elemento.canvas_id = self.canvas.create_rectangle(x, y + top_height, x + width, y + height, fill="lightcyan")
            self.canvas.create_oval(x, y, x + width, y + top_height*2, fill="lightcyan")
        
        # Adicionar texto do nome
        text_id = self.canvas.create_text(x + width/2, y + height/2, text=elemento.nome, width=width-10)
        
        # Atualizar as dimensões reais com base no texto
        bbox = self.canvas.bbox(text_id)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        
        # Garantir que o elemento seja grande o suficiente para o texto
        min_width = text_width + 20
        min_height = text_height + 20
        
        if min_width > width or min_height > height:
            new_width = max(width, min_width)
            new_height = max(height, min_height)
            
            # Atualizar as dimensões do elemento
            elemento.width = new_width
            elemento.height = new_height
            
            # Redesenhar o elemento com as novas dimensões
            self.canvas.delete(elemento.canvas_id)
            self.canvas.delete(text_id)
            self.desenhar_elemento(elemento)
    
    def desenhar_conexao(self, origem, destino):
        # Calcular pontos de conexão
        # Decidir de qual lado do elemento a conexão sai/entra
        
        # Coordenadas do centro dos elementos
        origem_x = origem.x + origem.width / 2
        origem_y = origem.y + origem.height / 2
        destino_x = destino.x + destino.width / 2
        destino_y = destino.y + destino.height / 2
        
        # Determinar os pontos de saída e entrada
        if origem.x + origem.width < destino.x:  # Origem à esquerda do destino
            # Saída pela direita da origem
            start_x = origem.x + origem.width
            start_y = origem_y
            
            # Entrada pela esquerda do destino
            end_x = destino.x
            end_y = destino_y
        elif origem.x > destino.x + destino.width:  # Origem à direita do destino
            # Saída pela esquerda da origem
            start_x = origem.x
            start_y = origem_y
            
            # Entrada pela direita do destino
            end_x = destino.x + destino.width
            end_y = destino_y
        elif origem.y + origem.height < destino.y:  # Origem acima do destino
            # Saída pela parte inferior da origem
            start_x = origem_x
            start_y = origem.y + origem.height
            
            # Entrada pela parte superior do destino
            end_x = destino_x
            end_y = destino.y
        else:  # Origem abaixo do destino
            # Saída pela parte superior da origem
            start_x = origem_x
            start_y = origem.y
            
            # Entrada pela parte inferior do destino
            end_x = destino_x
            end_y = destino.y + destino.height
        
        # Desenhar a linha de conexão
        # Para criar uma linha perpendicular, usamos pontos intermediários
        if abs(start_x - end_x) > abs(start_y - end_y):
            # Conexão horizontal predominante, adicionar ponto intermediário vertical
            mid_x = (start_x + end_x) / 2
            self.canvas.create_line(start_x, start_y, mid_x, start_y, mid_x, end_y, end_x, end_y, arrow=tk.LAST)
        else:
            # Conexão vertical predominante, adicionar ponto intermediário horizontal
            mid_y = (start_y + end_y) / 2
            self.canvas.create_line(start_x, start_y, start_x, mid_y, end_x, mid_y, end_x, end_y, arrow=tk.LAST)
    
    def gerar_descricao(self):
        if not self.elementos:
            messagebox.showerror("Erro", "Não há elementos no fluxograma para gerar uma descrição.")
            return
        
        # Encontrar elemento inicial
        inicio = None
        for elemento in self.elementos:
            if elemento.tipo == "Início":
                inicio = elemento
                break
        
        if not inicio:
            messagebox.showerror("Erro", "Não foi encontrado um elemento do tipo 'Início'.")
            return
        
        # Gerar descrição do fluxograma
        descricao = "Descrição do Fluxograma:\n\n"
        descricao += f"Este fluxograma inicia com '{inicio.nome}' e "
        
        # Contar tipos de elementos
        tipos_count = {}
        for elemento in self.elementos:
            if elemento.tipo in tipos_count:
                tipos_count[elemento.tipo] += 1
            else:
                tipos_count[elemento.tipo] = 1
        
        descricao += "contém:\n"
        for tipo, count in tipos_count.items():
            descricao += f"- {count} elemento(s) do tipo '{tipo}'\n"
        
        descricao += f"\nTotal de {len(self.elementos)} elementos e {len(self.conexoes)} conexões.\n\n"
        
        # Tentar traçar um caminho a partir do início
        visitados = set()
        caminho = []
        
        def trace_path(elemento):
            if elemento in visitados:
                return
            
            visitados.add(elemento)
            caminho.append(elemento)
            
            for proximo in elemento.conexoes:
                trace_path(proximo)
        
        trace_path(inicio)
        
        # Adicionar descrição do caminho
        if len(caminho) > 1:
            descricao += "Sequência de execução:\n"
            for i, elemento in enumerate(caminho):
                descricao += f"{i+1}. {elemento.nome} ({elemento.tipo})"
                if i < len(caminho) - 1:
                    descricao += " → "
                if (i + 1) % 3 == 0:  # Quebra de linha a cada 3 elementos
                    descricao += "\n"
        
        # Elementos não visitados (possivelmente desconectados)
        nao_visitados = [e for e in self.elementos if e not in visitados]
        if nao_visitados:
            descricao += "\n\nElementos não conectados ao fluxo principal:\n"
            for elemento in nao_visitados:
                descricao += f"- {elemento.nome} ({elemento.tipo})\n"
        
        # Decisões e suas ramificações
        decisoes = [e for e in self.elementos if e.tipo == "Decisão"]
        if decisoes:
            descricao += "\nPontos de decisão:\n"
            for decisao in decisoes:
                descricao += f"- '{decisao.nome}' pode levar a: "
                destinos = [f"'{dest.nome}'" for origem, dest in self.conexoes if origem == decisao]
                if destinos:
                    descricao += ", ".join(destinos)
                else:
                    descricao += "nenhum destino conectado"
                descricao += "\n"
        
        # Atualizar a área de texto
        self.descricao_text.delete(1.0, tk.END)
        self.descricao_text.insert(tk.END, descricao)
    
    def limpar_tudo(self):
        if messagebox.askyesno("Confirmar", "Tem certeza que deseja limpar todos os elementos do fluxograma?"):
            self.canvas.delete("all")
            self.elementos = []
            self.conexoes = []
            self.ultimo_elemento = None
            self.descricao_text.delete(1.0, tk.END)

if __name__ == "__main__":
    app = SistemaFluxograma()
    app.mainloop()